In [1]:
import pandas as pd
import numpy as np
import gzip 
import ast
import pyarrow as pa
import pyarrow.parquet as pq

Ingesta de datos

In [2]:
datos_json3 = []
# Abrir el archivo y leer línea por línea
with gzip.open('../json/users_items.json.gz', 'rt', encoding= 'utf-8') as f3:
    for li in f3:
        json_linea2 = ast.literal_eval(li)
        # Agregar el objeto JSON decodificado a la lista
        datos_json3.append(json_linea2)

Ingesta de datos

In [3]:
user_items_df = pd.DataFrame(datos_json3)

In [4]:
print(user_items_df.info())
user_items_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB
None


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [5]:
user_items_df.dropna(how='all', inplace=True) # 88310 entries
# Elimino nulos

In [6]:
def desanidar_items(row):
    list = []
    for item in row['items']:
        data = {
            'user_id': row['user_id'],
            'steam_id': row['steam_id'],
            **item  # Añade los atributos del elemento como nuevas columnas
        }
        list.append(data)
    return pd.DataFrame(list)

In [7]:
items_df = pd.concat(user_items_df.apply(desanidar_items, axis=1).tolist(), ignore_index=True)

In [8]:
items_df = items_df.drop(columns='playtime_2weeks')

In [9]:
items_df['item_id'] = items_df['item_id'].astype(int)
items_df['playtime_forever'] = items_df['playtime_forever'].astype(int)


In [10]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   steam_id          object
 2   item_id           int32 
 3   item_name         object
 4   playtime_forever  int32 
dtypes: int32(2), object(3)
memory usage: 157.3+ MB


In [11]:
items_df = items_df.drop(columns=['steam_id'])

In [15]:
items_df_parts = np.array_split(items_df, 10)
for i, part in enumerate(items_df_parts):
    table = pa.Table.from_pandas(part)
    pq.write_table(table, f"../data_transformed/parts_user_items/part_user_items{i}.parquet", compression='snappy')

In [ ]:
table = pa.Table.from_pandas(items_df)
pq.write_table(table, "../data_transformed/user_items.parquet", compression='snappy')

In [ ]:
data = pq.read_table("../data_transformed/user_items.parquet")
user_df = data.to_pandas()
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           int32 
 2   item_name         object
 3   playtime_forever  int32 
dtypes: int32(2), object(2)
memory usage: 117.9+ MB
